In [1]:
%load_ext line_profiler

ModuleNotFoundError: No module named 'line_profiler'

In [1]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from k3d.platonic import PlatonicSolid
from itertools import product
import qutip as qt
from scipy.linalg import expm

In [2]:
from k3d.platonic import PlatonicSolid
from itertools import product

class Box(PlatonicSolid):
    def __init__(self, origin=[0, 0, 0], size=[1,1,1]):
        origin = np.array(origin, dtype=np.float32)
        size = np.array(size, dtype=np.float32)
        
        if origin.shape == (3,):
            cube_vertices = np.array(list(product([size[0], -size[0]], [size[1], -size[1]], [size[2], -size[2]])), np.float32)
            cube_vertices = np.float32(cube_vertices + origin)

            self.vertices = cube_vertices
            self.indices = [0, 1, 2, 1, 2, 3, 0, 1, 4, 1, 4, 5, 1, 3, 5, 3, 5, 7, 0, 2, 4, 2, 4, 6, 2, 3, 7, 2, 6, 7, 4,
                            5, 6, 5, 6, 7]

        else:
            raise TypeError('Origin attribute should have 3 coordinates.')

In [3]:
def getrot(a,b):
    v = np.cross(a, b)
    s = np.linalg.norm(v)
    c = np.dot(a,b)
    vx = np.array([[0,-v[2], v[1]],[v[2], 0, v[0]], [-v[1], v[0], 0]])
    return np.identity(3) + vx + vx@vx*(1/(1+c))#(1-c)/s**2

def pad3to4(mat):
    m = np.identity(4)
    m[0:3,0:3] = mat
    return m

def norm(arr, axis=-1):
    norm = np.sqrt(np.sum(arr**2, axis=-1))
    norm = np.expand_dims(norm, axis)
    return np.divide(arr, norm, where=(norm!=0))

In [4]:
import itertools

class InstanceCounterMeta(type):
    #Metaclass to make instance counter not share count with descendants
    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        cls._ids = itertools.count(1)

class Optic(object, metaclass=InstanceCounterMeta):
    def __init__(self, m=None):
        self.id = next(self.__class__._ids)
        self.m = np.identity(4, dtype=np.float64)
    
    def __matmul__(self, other):
        return Optic(np.dot(self.m,other.m))

class Prop(Optic, metaclass=InstanceCounterMeta):
    def __init__(self, d):
        self.id = next(self.__class__._ids)
        self.dist = d
        m = np.identity(4, dtype=np.float64)
        m[0,2] = d
        m[1,3] = d
        self.m = m
    
    def _aberrations(self, chi):
        #shape of chi is (4, N, N)
        x, y, sx, sy = chi
        sx2, sy2 = sx@sx, sy@sy
        sx2psy2 = sx2+sy2
        nonP4 = -self.dist*k/8*(sx2psy2@sx2psy2)
        nonP6 = -self.dist*k/16*(sx2psy2@sx2psy2@sx2psy2)
        #nonP8 = -5*self.dist*k/128*(sx2psy2@sx2psy2@sx2psy2@sx2psy2)
        
        return [nonP4, nonP6] #nonP6, nonP8 
        
    def aberration(self, chi):
        return sum(self._aberrations(chi))
    
class Rot(Optic, metaclass=InstanceCounterMeta):
    def __init__(self, r):
        self.id = next(self.__class__._ids)
        self.r = r
        m = np.zeros((4,4), dtype=np.float64)
        m[0:2,0:2] = r
        m[2:4,2:4] = r
        self.m = m
    
class Lens(Optic, metaclass=InstanceCounterMeta):
    def __init__(self, r1, r2=None, thet=0., n=1.5):
        self.id = next(self.__class__._ids)
        #f = With[{R1 = \[Infinity], R2 = R}, 1/(n - 1) (1/R1 + 1/R2)^-1];
        
        if r2 is None:
            f = r1/(n-1)
        else:
            f = 1./((n-1)*(1/r1-1/r2))
        
        self.r = r1
        self.f = f
        self.n = n
        
        self.thet = thet
        self.cos = np.cos(thet)
        self.sin = np.sin(thet)
        
        m = np.identity(4, dtype=np.float64)
        m[2,0] = -1./(f*self.cos)
        m[3,1] = -1./f*self.cos
        self.m = m
        
    def _aberrations(self, chi):
        #shape of chi is (4, N, N)
        x, y, sx, sy = chi
        x2, y2 = x@x, y@y
        x2py2 = x2+y2
        
        sph3 = 0.5*self.sin/(self.cos**3*self.r**2)*(x2+self.cos**2*y2)@x
        sph4 = 0.5/(4*self.cos**5*self.r**3)*(self.cos**2*y2+x2)@(-4*self.cos**2*x2+self.cos**2*y2+5*x2)
        sph5 = 0.5*self.sin/(4*self.cos**7*self.r**4)*(self.cos**2*y2+x2)@(3*self.cos**2*x@y2+(7-4*self.cos**2)*x@x2)
        sph6 = 0.5/(8*self.cos**9*self.r**5)*(self.cos**2*y2+x2)@(self.cos**4*y2@y2+(14*self.cos**2-12*self.cos**4)*x2@y2+(8*self.cos**4-28*self.cos**2+21)*x2@x2)
        return [sph3, sph4, sph5, sph6] #
        
    def aberration(self, chi):
        return k*(1-self.n)*sum(self._aberrations(chi))
        
class Mirr(Lens, metaclass=InstanceCounterMeta):
    def __init__(self, roc, thet=0.):
        super().__init__(n=3., r1 = roc, thet = thet)
        #self.id = next(self.__class__._ids)
        #self.roc = roc

$$\text{sphAb3rd}(\text{R$\_$},\text{Cos$\theta \_$})=\frac{1}{2} \sqrt{1-\text{Cos$\theta $}^2} \frac{\text{Cos$\theta $}^2 y.y+x.x}{\text{Cos$\theta $}^3 R^2}.x;$$
$$\text{sphAb4th}(\text{R$\_$},\text{Cos$\theta \_$})=\frac{1}{2} \frac{\text{Cos$\theta $}^2 y.y+x.x}{4 \text{Cos$\theta $}^5 R^3}.\left(-4 \text{Cos$\theta $}^2 x.x+\text{Cos$\theta $}^2 y.y+5 x.x\right); $$
$$ \text{sphAb5th}(\text{R$\_$},\text{Cos$\theta \_$})=\frac{1}{2} \sqrt{1-\text{Cos$\theta $}^2} \frac{\text{Cos$\theta $}^2 y^2+x^2}{4 \text{Cos$\theta $}^7 R^4}.\left(3 \text{Cos$\theta $}^2 x.y.y+\left(7-4 \text{Cos$\theta $}^2\right) x.x.x\right);$$
$$ \text{sphAb6th}(\text{R$\_$},\text{Cos$\theta \_$})=\frac{1}{2} \frac{\text{Cos$\theta $}^2 y.y+x.x}{8 \text{Cos$\theta $}^9 R^5}.\left(\text{Cos$\theta $}^4 y.y.y.y+\left(14 \text{Cos$\theta $}^2-12 \text{Cos$\theta $}^4\right) x.x.y.y+\left(8 \text{Cos$\theta $}^4-28 \text{Cos$\theta $}^2+21\right) x.x.x.x\right);$$
$$ \text{nonParax4th}(\text{d$\_$})=\frac{1}{8} (-(d k)) (\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}); $$
$$ \text{nonParax6th}(\text{d$\_$})=\frac{1}{16} (-(d k)) (\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}); $$
$$ \text{nonParax8th}(\text{d$\_$})=\frac{1}{128} (-(5 d k)) (\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}).(\text{sx}.\text{sx}+\text{sy}.\text{sy}); $$

In [5]:
def Tetrahedron(La, thet, roc):
    
    if not isinstance(roc, list):
        roc = [roc]*4
        
    thet = np.deg2rad(thet)
    mir = np.array([(-La/2, La*np.tan(thet), 0),\
                ( La/2, 0, -La*np.tan(thet)),\
                (-La/2, -La*np.tan(thet), 0),\
                ( La/2, 0,  La*np.tan(thet))], dtype=np.float64)
    return {'mir': mir, 'roc': roc}

In [6]:
def OriginalTwister(betal, R=0.025, Rlarge=-0.075, thet=10., asym = 1.25):
    thet = np.deg2rad(thet)
    l = betal*R
    d1 = l*np.tan(thet/2)
    d2 = d1
    p1 = np.array([-d1, 0, -l/2] , dtype=np.float64)
    p2 = np.array([0, -d2, l/2] , dtype=np.float64)/asym
    p3 = np.array([d1, 0, -l/2] , dtype=np.float64)/asym
    p4 = np.array([0, d2, l/2] , dtype=np.float64)
    
    ax = 0.5*(p2+p3) - 0.5*(p1+p4)
    p2 += 0.25*ax
    p3 += 0.25*ax
    
    mir = np.stack([p1,p2,p3,p4], axis=0)
    roc = np.array([R, Rlarge, Rlarge, R])
    #mir = np.stack([p2,p3,p4,p1], axis=0)
    #roc = np.array([Rlarge, Rlarge, R, R])
    return {'mir': mir, 'roc': roc}

In [7]:
def SixMirror(thetOpening = 20, Rfast = 50e-3, Rplane = 1e5, Rslow = -250e-3, diam=6e-3, dy=8e-3, dx=6.0e-2):
    thetOpening = np.deg2rad(thetOpening)
    dz = np.tan(thetOpening)*np.sqrt(dx**2+dy**2)
    d = 1.31762e-2
    dzF = 1.0625e-3
    dzF2 = 0
    ddy = 0.
    
    p1=np.array([0,0,0], dtype=np.float64)
    p2=np.array([dx,dy,dzF+dzF2], dtype=np.float64)
    p3=np.array([0,dy,dzF ], dtype=np.float64)
    p4=np.array([dx,dy+dy + ddy,0], dtype=np.float64)
    p5=np.array([0+d,dy+d,dz], dtype=np.float64)
    p6=np.array([dx-d,dy-d,dz], dtype=np.float64)
    
    mir = np.stack([p3,p4,p5,p6,p1,p2], axis=0)
    roc = np.array([Rfast, Rplane, Rslow, Rslow, Rplane, Rfast])

    return {'mir': mir, 'roc': roc}

In [8]:
def geometry(mir, roc):
    Nm = len(mir)
    M = mir[:,None,:]-mir[None,:,:]
    m = norm(M)
    
    n = norm(np.array([np.cross(m[j,j-1],m[j,(j+1)%Nm]) for j in range(Nm)]))
    refl = norm(np.array([0.5*(m[j,j-1]+m[j,(j+1)%Nm]) for j in range(Nm)])) #vectors normal to reflecting mirrors
    angles = np.array([0.5*np.arccos(np.dot(m[j,j-1],m[j,(j+1)%Nm])) for j in range(Nm)])
    yin = n
    yout = n
    xin = norm(np.array([np.cross(n[j],m[j,j-1]) for j in range(Nm)]))
    xout = norm(np.array([np.cross(n[j],-m[j,(j+1)%Nm]) for j in range(Nm)]))
    
    R = np.stack([np.array([[xout[i]@xin[(i+1)%Nm], yout[i]@xin[(i+1)%Nm]],\
                            [xout[i]@yin[(i+1)%Nm], yout[i]@yin[(i+1)%Nm]]]) for i in range(Nm)], axis=0)
    #Reflect vector
    def RefV(j, vec):
        return vec - ((vec@m[j,(j+1)%Nm])*(m[j,(j-1)%Nm]+m[j,(j+1)%Nm])/(1.+m[j,(j-1)%Nm]@m[j,(j+1)%Nm]))
    
    changeBasisAcross = np.stack([np.array([[RefV(i,xin[i])@xout[i], RefV(i,yin[i])@xout[i]],\
                                            [RefV(i,xin[i])@yout[i], RefV(i,yin[i])@yout[i]]]) for i in range(Nm)], axis=0)
    
    changeBasisAcross = np.round(changeBasisAcross) # fix slight numerical errors of the reflection
    
    #Build the ABCD Matrix
    Ls = [np.linalg.norm(M[j-1,j]) for j in range(Nm)]
    Lrt = sum(Ls)
    
    
    AllM = []
    for i in range(Nm):
        d1 = np.linalg.norm(M[i-1,i])/2
        d2 = np.linalg.norm(M[i,(i+1)%Nm])/2
        
        #Msub = [Prop(d1), Mirr(roc[i], angles[i]), Rot(changeBasisAcross[i]), Rot(R[i]), Prop(d2)]
        Msub = [Prop(d1), Lens(r1=roc[i], thet=angles[i], n=3.), Rot(changeBasisAcross[i]), Rot(R[i]), Prop(d2)]
        AllM.extend(Msub)
        
    Mrt = np.identity(4, dtype=np.float64)
    for i, M in enumerate(AllM):
        Mrt = np.dot(M.m,Mrt)
        
    geom = {'mir': mir, 'Ls': Ls, 'Lrt': Lrt, 'm': m, 'n': n, 'refl': refl, 'xin': xin, 'xout': xout}
    return {'M': Mrt, 'AllM': AllM, 'geometry': geom}

In [9]:
def plot_geometry(mir, m, n, refl, xin, xout, **kwargs):
    Nm = len(mir)
    plot = k3d.plot(camera_auto_fit=True, antialias=True)

    col=0xff0000
    pf = 200.
    plt_line = k3d.line(pf*mir, shader='mesh', width=0.1, color=col)
    plt_line2 = k3d.line(pf*mir[(-1,0),...], shader='mesh', width=0.1, color=col)
    plot += plt_line
    plot += plt_line2
    plot += k3d.vectors(origins=pf*mir, vectors=n*2, use_head=True, head_size=3.)#Normals = xIn = xOut
    plot += k3d.vectors(origins=pf*mir, vectors=xin*2, use_head=True, head_size=3., color= 0xff8c00) #yIn
    plot += k3d.vectors(origins=pf*mir, vectors=xout*2, use_head=True, head_size=3., color= 0x9400D3) #yOut
    plot += k3d.vectors(origins=pf*mir, vectors=refl*2, use_head=True, head_size=3., color=0x00ff00)

    ey = np.array([0,1,0])
    ex = np.array([1,0,0])
    for i in range(Nm):
        mirror = Box(size=(0.1,.5,.5)).mesh

        mirror.transform.custom_matrix = pad3to4(getrot(ex, refl[i])) #get rotation matrix of mirror
        mirror.transform.translation = pf*mir[i]
        plot += mirror

    return plot

In [10]:
G = np.array([[0,0,1,0], [0,0,0,1], [-1,0,0,0], [0,-1,0,0]])

def get_conj_pairs(ev):
    ind = [0, 1, 2, 3]
    indout = []
    evout = []
    while len(ind)>0:
        i = ind.pop(0)
        j = ind[np.argsort(np.abs(ev[i].conj()-ev[ind]))[0]]
        if ev[i].imag<ev[j].imag:
            i, j = j, i
        indout.append(i)
        evout.append(ev[i])
        ind.remove(j)
    return indout, evout

    

def M2waist(Mrt, lam):
    ev, mus = np.linalg.eig(Mrt)

    #Normalize the eigenvectors and take the ones that are normalized to -2j
    N = [mus[:,i].conj()@G@mus[:,i] for i in range(4)]
    musN = np.stack([mus[:,i]*np.sqrt(2/N[i]) for i in range(4)], axis=1)
    N = np.array([musN[:,i].conj()@G@musN[:,i] for i in range(4)])
    #print(N)
    idx = np.where(np.abs(N.imag+2)<1e-5)[0]
    idx = idx[::-1]
    musN = musN[:,idx]
    #Build B and K matrix, this is very sensitive! Double check again, see https://journals.aps.org/pra/pdf/10.1103/PhysRevA.75.033819
    B = np.array([[ musN[0,0], musN[1,0] ],[ musN[0,1], musN[1,1] ]])
    K = -1j*np.array([[ musN[2,0], musN[3,0] ],[ musN[2,1], musN[3,1] ]])
    # Q = B^-1 K
    Q = np.linalg.solve(B,K) 
    di = np.linalg.eigvals(1j*Q)
    ws = np.sqrt(lam/(-np.pi*di.imag))
    return {'ws': ws, 'B': B, 'K': K, 'Q': Q, 'mu': musN}

from math import floor
def wrap_fsr(fsr, f):
    return f%(fsr/2) - fsr/2*( 1 if floor(2*f/fsr)%2==1 else 0 )

def M2freq(Mrt, geom):
    fsr = 3.0e8/geom['Lrt']
    ev = np.linalg.eigvals(Mrt)
    freqs = np.angle(ev)*fsr/(2*np.pi)
    freqsA = np.sort(freqs)[-1:-3:-1]
    freqsB = freqsA - fsr
    freqsC = 3.*freqsA - fsr
    return freqsA, freqsC

In [11]:
def M2waistNew(Mrt, lam):
    ev, mus = np.linalg.eig(Mrt)

    #Normalize the eigenvectors and take the ones that are normalized to -2j
    N = [mus[:,i].conj()@G@mus[:,i] for i in range(4)]
    musN = np.stack([mus[:,i]*np.sqrt(2/N[i]) for i in range(4)], axis=1)
    N = np.array([musN[:,i].conj()@G@musN[:,i] for i in range(4)])
    #print(N)
    idx = np.where(np.abs(N.imag+2)<1e-5)[0]
    idx = idx[::-1]
    musN = musN[:,idx]
    #Build B and K matrix, this is very sensitive! Double check again, see https://journals.aps.org/pra/pdf/10.1103/PhysRevA.75.033819
    B = np.array([[ musN[0,0], musN[1,0] ],[ musN[0,1], musN[1,1] ]])
    K = -1j*np.array([[ musN[2,0], musN[3,0] ],[ musN[2,1], musN[3,1] ]])
    # Q = B^-1 K
    Q = np.linalg.solve(B,K) 
    di = np.linalg.eigvals(1j*Q)
    ws = np.sqrt(lam/(-np.pi*di.imag))
    return {'ws': ws, 'B': B, 'K': K, 'Q': Q, 'mu': musN}

In [12]:
lamb = 780.0e-9
k = 2*np.pi/lamb

#opt = Tetrahedron(La = 5.1e-2, thet = 20, roc=100e-3)
#opt = OriginalTwister(betal=1.31323, R=0.025, Rlarge=-0.075, thet=20., asym = 1.25)#betal=1.35
opt = SixMirror(thetOpening = 20, Rfast = 50e-3, Rplane = 1e5, Rslow = -250e-3, diam=6e-3, dy=8e-3, dx=6.0e-2)
system = geometry(**opt)
plot_geometry(**system['geometry'])

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [13]:
Mrt = system['M']

In [14]:
ev, mus = np.linalg.eig(Mrt)

#Normalize the eigenvectors and take the ones that are normalized to -2j
N = [mus[:,i].conj()@G@mus[:,i] for i in range(4)]
musN = np.stack([mus[:,i]@G.T/np.sqrt(-1j*N[i]) for i in range(4)], axis=1)
N = np.array([musN[:,i].conj()@G@musN[:,i] for i in range(4)])

In [15]:
N

array([8.88162212e-20-1.j, 8.88162212e-20+1.j, 3.78381979e-19-1.j,
       3.78381979e-19+1.j])

In [16]:
#N = np.array([musN[:,i].conj()@G@musN[:,i] for i in range(4)])
#print(N)
idx = np.where(np.abs(N.imag-1)<1e-5)[0]
idx = idx[::-1]
print(idx)
musN = musN[:,idx]
#Build B and K matrix, this is very sensitive! Double check again, see https://journals.aps.org/pra/pdf/10.1103/PhysRevA.75.033819
V = np.array([[ musN[0,0], musN[1,0] ],[ musN[0,1], musN[1,1] ]])
U = np.array([[ musN[2,0], musN[2,1] ],[ musN[3,0], musN[3,1] ]])

[3 1]


In [17]:
W = 1j*V@np.linalg.inv(U)
W

array([[745.02771385-20.80759589j, -32.03669586+22.51654587j],
       [ 20.91504073 +0.63550949j, 719.92803255+20.71137238j]])

## Realspace

In [18]:
wmat = M2waist(system['M'], lamb)
Q = wmat['Q']
waist = wmat['ws'][0]
sgn = 1 if np.trace(Q).real>0 else -1
print(sgn)

xx=np.linspace(-5*waist,5*waist,200)
dx = xx[1]-xx[0]
x, y = np.meshgrid(xx,xx, indexing='ij')
r = np.dstack([x,y])
r.shape

-1


(200, 200, 2)

In [19]:
def get_phi0(r, Q):
    phi0 = np.exp(-sgn*0.5*k*np.einsum("ijk,ijl,kl->ij",r,r,Q))
    phi0 *= 1./np.sqrt(np.linalg.norm(np.abs(phi0)**2))
    #np.linalg.norm(np.abs(phi0)**2)
    return phi0

In [20]:
phi0 = get_phi0(r, Q)

In [21]:
def chi_wf(phi):
    pdx = np.gradient(phi, dx, axis=0)
    pdy = np.gradient(phi, dx, axis=1)
    px = phi*x
    py = phi*y
    return np.stack([px, py, 1j*pdx/k, 1j*pdy/k], axis=0)

def a_op(phi, mu, dag=False):
    if dag:
        pre = -1j*np.sqrt(k/2)*np.conj(mu)
    else:
        pre = 1j*np.sqrt(k/2)*mu
    return np.einsum("i,ij,jxy->xy",pre,G,chi_wf(phi))

def chain(inp, funcs):
    return reduce(lambda x, y : y(x), funcs, inp)
    
def nplt(phi, ax):
    return ax.imshow(np.abs(phi.T)**2, origin='lower', interpolation='bicubic')

In [22]:
ra = lambda x: a_op(x, wmat['mu'][:,0], dag=False)
raD = lambda x: a_op(x, wmat['mu'][:,0], dag=True)
rb = lambda x: a_op(x, wmat['mu'][:,1], dag=False)
rbD = lambda x: a_op(x, wmat['mu'][:,1], dag=True)

In [23]:
plt.figure()
plt.imshow(np.abs(chain(phi0, [rbD]))**2)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
Na = 5
Nb = 5
phis = np.empty((Na, Nb, *phi0.shape), dtype=np.complex)
phis[0,0] = phi0
for i in range(1,Na):
    phis[i,0] = raD(phis[i-1,0])/np.sqrt(i)
for i in range(Na):
    for j in range(1,Nb):
        phis[i,j] = rbD(phis[i,j-1])/np.sqrt(j)

In [25]:
from mpl_toolkits.axes_grid1 import AxesGrid

fig = plt.figure(figsize=(Na, Nb))
grid = AxesGrid(fig, 111, nrows_ncols=(Na, Nb), axes_pad=0.05, cbar_mode='single', cbar_location='right', cbar_pad=0.1)

for i, row in enumerate(grid.axes_row):
    for j, ax in enumerate(row):
        ax.set_axis_off()
        im = ax.imshow(np.abs(phis[i,j])**2, origin='lower', interpolation='bicubic')

cbar = ax.cax.colorbar(im)
cbar = grid.cbar_axes[0].colorbar(im)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  del sys.path[0]


In [26]:
N1 = 5
N2 = N1**2
phis1 = np.empty((N2, *phi0.shape), dtype=np.complex)
phis1[0] = phi0
for i in range(1,N2):
    phis1[i] = raD(phis1[i-1])/np.sqrt(i)
        
fig = plt.figure(figsize=(N1, N1))
grid = AxesGrid(fig, 111, nrows_ncols=(N1, N1), axes_pad=0.05, cbar_mode='single', cbar_location='right', cbar_pad=0.1)

for i, row in enumerate(grid.axes_row):
    for j, ax in enumerate(row):
        ax.set_axis_off()
        im = ax.imshow(np.abs(phis1[i*N1+j])**2, origin='lower', interpolation='bicubic')

cbar = ax.cax.colorbar(im)
cbar = grid.cbar_axes[0].colorbar(im)
#plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Aberration calculation

In [27]:
def getEvN(Mrt):
    ev, mus = np.linalg.eig(Mrt)
    
    #Normalize the eigenvectors and take the ones that are normalized to -2j
    N = [mus[:,i].conj()@G@mus[:,i] for i in range(4)]
    musN = np.stack([mus[:,i]*np.sqrt(2/N[i]) for i in range(4)], axis=1)
    #print(musN)
    N = np.array([musN[:,i].conj()@G@musN[:,i] for i in range(4)])
    #print(N)
    idx = np.where(np.abs(N.imag-2)<1e-5)[0]
    #print(N, idx)
    #idx = idx[::-1]
    musN = musN[:,idx]
    Us = ev[idx]
    chis = np.angle(Us)
    order = np.argsort(chis)[::-1] #have the ev with the smallest chi first
    return {'mu': musN[:,order], 'U': Us[order], 'chi': chis[order]}

In [28]:
np.set_printoptions(precision=3)

In [29]:
MuAndChi = getEvN(system['M'])

In [30]:
Na, Nb = 50, 50
a = qt.tensor(qt.destroy(Na), qt.qeye(Nb))
b = qt.tensor(qt.qeye(Na), qt.destroy(Nb))
aD = a.dag()
bD = b.dag()

ops_sp = [a, b, aD, bD]

xsi1, xsi2 = MuAndChi['chi']

#turn into numpy array for vectorized operations
a = np.array(a.data.todense(), dtype=np.complex)
b = np.array(b.data.todense(), dtype=np.complex)
aD = np.array(aD.data.todense(), dtype=np.complex)
bD = np.array(bD.data.todense(), dtype=np.complex)

#Upar = expm(1j*(xsi1*aD@a + xsi2*bD@b))
chi = np.stack([a, b, aD, bD], axis=0)

In [31]:
mu = MuAndChi['mu']
InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))

In [32]:
XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, InvEigs.T, chi)
XPrefMat.shape

(4, 2500, 2500)

In [33]:
Utot = Upar
Mcum = np.eye(4)
for i, M in enumerate(system['AllM']):
    Mcum = np.dot(M.m, Mcum) #ABCD matrix up to current element
    #only add aberrations after Prop and Lens!
    if isinstance(M, Prop) or isinstance(M, Lens):
        XP = np.einsum("ij,jxy->ixy",Mcum,XPrefMat)
        pert = M.aberration(XP)
        Utot = expm(1j*pert)@Utot

NameError: name 'Upar' is not defined

In [118]:
np.allclose(np.abs(XP), np.abs(XPrefMat))

True

In [148]:
import sparse
from opt_einsum import contract, contract_expression

In [192]:
coo_data = np.concatenate([ops_sp[i].data.tocoo().data for i in range(4)])

coo_row = np.concatenate([ops_sp[i].data.tocoo().row for i in range(4)])
coo_col = np.concatenate([ops_sp[i].data.tocoo().col for i in range(4)])
coo_page = np.concatenate([np.ones_like(ops_sp[i].data.tocoo().row)*i for i in range(4)])

coo_cords = np.stack([coo_page, coo_row, coo_col], axis=0)
coo_cords.shape

chi_sp = sparse.COO(coo_cords, data=coo_data, shape=(4, *ops_sp[0].shape), fill_value=0.)

In [207]:
chi_sp.todense()

array([[[0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        ...,
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j]],

       [[0.   +0.j, 1.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 1.414+0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
         0.   +0.j],
        ...,
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 6.928+0.j,
         0.   +0.j],
        [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
   

In [124]:
np.allclose(chi_sp.todense(), chi)

True

In [135]:
G_sp = sparse.COO.from_numpy(G)
InvEigs_sp = sparse.COO.from_numpy(InvEigs.T)

In [136]:
XPrefMat_sp = np.sqrt(2/k)*contract("ji,jk,kxy->ixy", G_sp, InvEigs_sp, chi_sp, backend='sparse')

In [142]:
XPrefMat_sp

Format,coo
Data Type,complex128
Shape,"(4, 100, 100)"
nnz,1440
Density,0.036
Read-only,True
Size,56.2K
Storage ratio,0.1


In [154]:
#XP_sp = contract("ij,jxy->ixy",sparse.COO.from_numpy(Mcum), XPrefMat_sp, backend='sparse')
XP_expr = contract_expression("ij,jxy->ixy", (4, 4), XPrefMat_sp.shape)

In [158]:
%timeit XP_expr(sparse.COO.from_numpy(Mcum), XPrefMat_sp, backend='sparse')

1000 loops, best of 5: 822 µs per loop


In [159]:
%timeit XP = np.einsum("ij,jxy->ixy",Mcum,XPrefMat)

1000 loops, best of 5: 532 µs per loop


In [161]:
XP_sp = XP_expr(sparse.COO.from_numpy(Mcum), XPrefMat_sp, backend='sparse')

In [165]:
XP_sp@XP_sp

Format,coo
Data Type,complex128
Shape,"(4, 100, 100)"
nnz,2976
Density,0.0744
Read-only,True
Size,116.2K
Storage ratio,0.2


In [197]:
from scipy.sparse.linalg import expm as expm_sp

In [201]:
expm_sp

<function scipy.sparse.linalg.matfuncs.expm>

In [208]:
def calcPertSparse(system, chi):
    a, b, aD, bD = chi
    MuAndChi = getEvN(system['M'])
    xsi1, xsi2 = MuAndChi['chi']
    
    mu = MuAndChi['mu']
    InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
    G_sp = sparse.COO.from_numpy(G)
    InvEigs_sp = sparse.COO.from_numpy(InvEigs.T)
    
    Argpar = (xsi1*aD@a + xsi2*bD@b) # argument of Upar = expm(1j*Argpar), is diagonal anyways!
    #XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, InvEigs.T, chi)
    XPrefMat_sp = np.sqrt(2/k)*contract("ji,jk,kxy->ixy", G_sp, InvEigs_sp, chi, backend='sparse')
    
    XP_expr = contract_expression("ij,jxy->ixy", (4, 4), XPrefMat_sp.shape)
    
    #Utot = Upar.copy()
    Argtot = Argpar.copy()
    Mcum = np.eye(4)
    for i, M in enumerate(system['AllM']):
        Mcum = np.dot(M.m, Mcum) #ABCD matrix up to current element
        #only add aberrations after Prop and Lens!
        if isinstance(M, Prop) or isinstance(M, Lens):
            #XP = np.einsum("ij,jxy->ixy",Mcum,XPrefMat)
            XP_sp = XP_expr(sparse.COO.from_numpy(Mcum), XPrefMat_sp, backend='sparse')
            pert = M.aberration(XP_sp)
            Argtot = Argtot + pert
            #Utot = expm(1j*pert)@Utot
    #Argscipy = Argtot.to_scipy_sparse().tocsc()        
    #Utot = expm_sp(1j*Argscipy)
    Argscipy = Argtot.todense()
    Utot = expm(1j*Argscipy)
    return np.sort(np.angle(np.linalg.eigvals(Utot))), np.sort(np.angle(np.exp(1j*np.diag(Argpar.todense()))))

In [182]:
np.allclose(calcPertFast(system, chi)[0], calcPertSparse(system, chi_sp)[0])

(100, 100)


True

In [180]:
def calcPert(system, chi):
    a, b, aD, bD = chi
    MuAndChi = getEvN(system['M'])
    xsi1, xsi2 = MuAndChi['chi']
    
    mu = MuAndChi['mu']
    InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
    Upar = expm(1j*(xsi1*aD@a + xsi2*bD@b))
    XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, InvEigs.T, chi)
    
    Utot = Upar.copy()
    Mcum = np.eye(4)
    for i, M in enumerate(system['AllM']):
        Mcum = np.dot(M.m, Mcum) #ABCD matrix up to current element
        #only add aberrations after Prop and Lens!
        if isinstance(M, Prop) or isinstance(M, Lens):
            XP = np.einsum("ij,jxy->ixy",Mcum,XPrefMat)
            pert = M.aberration(XP)
            Utot = expm(1j*pert)@Utot
    return np.sort(np.angle(np.linalg.eigvals(Utot))), np.sort(np.angle(np.linalg.eigvals(Upar)))

In [181]:
def calcPertFast(system, chi):
    a, b, aD, bD = chi
    MuAndChi = getEvN(system['M'])
    xsi1, xsi2 = MuAndChi['chi']
    
    mu = MuAndChi['mu']
    InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
    Argpar = (xsi1*aD@a + xsi2*bD@b) # argument of Upar = expm(1j*Argpar), is diagonal anyways!
    XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, InvEigs.T, chi)
    
    #Utot = Upar.copy()
    Argtot = Argpar.copy()
    Mcum = np.eye(4)
    for i, M in enumerate(system['AllM']):
        Mcum = np.dot(M.m, Mcum) #ABCD matrix up to current element
        #only add aberrations after Prop and Lens!
        if isinstance(M, Prop) or isinstance(M, Lens):
            XP = np.einsum("ij,jxy->ixy",Mcum,XPrefMat)
            pert = M.aberration(XP)
            Argtot = Argtot + pert
            #Utot = expm(1j*pert)@Utot
    Utot = expm(1j*Argtot)
    return np.sort(np.angle(np.linalg.eigvals(Utot))), np.sort(np.angle(np.exp(1j*np.diag(Argpar))))

In [56]:
np.allclose(calcPertFast(system, chi)[0], calcPert(system, chi)[0])

False

In [41]:
stab = lambda m: abs(0.5*np.trace(m))<1

N = 500
L0 = 5.86e-2#1.267
delt = 2e-4
#Las = np.linspace(L0-delt, L0+delt, N)
Las = np.linspace(5.858e-2, 5.8956e-2, N)

ms = np.zeros((N,1))
ws = np.zeros((N,2))
freqs = np.zeros((N,4))
for i, l in enumerate(Las):
    #system = construct(La = l, thet = 10, roc=[100e-3,]*4)
    #opt = OriginalTwister(betal=l, R=0.025, Rlarge=-0.075, thet=20., asym = 1.25)
    opt = SixMirror(thetOpening = 20, Rfast = 50e-3, Rplane = 1e5, Rslow = -250e-3, diam=6e-3, dy=8e-3, dx=l)
    system = geometry(**opt)
    ms[i] = stab(system['M'])
    w = M2waist(system['M'], lamb)
    f = M2freq(system['M'], system['geometry'])
    ws[i,:] = np.sort(w['ws'])
    freqs[i,...] = np.concatenate(f)

In [42]:
g, ax = plt.subplots(ncols=2, figsize=(8,4))
ax[0].plot(Las, ws*1e6)
ax[0].set_ylabel('um')
ax[1].plot(Las, freqs*1e-6)
ax[1].set_ylabel('MHz')
ax[1].axhline(0, color='grey')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
specPert = np.zeros((N, Na*Nb))
spec = np.zeros((N, Na*Nb))
fsrs =  np.zeros(N)
for i, l in enumerate(Las):
    #opt = OriginalTwister(betal=l, R=0.025, Rlarge=-0.075, thet=20., asym = 1.25)
    opt = SixMirror(thetOpening = 20, Rfast = 50e-3, Rplane = 1e5, Rslow = -250e-3, diam=6e-3, dy=8e-3, dx=l)
    system = geometry(**opt)
    specPert[i,:], spec[i,:] = calcPertFast(system, chi)
    fsrs[i] = 3.0e8/system['geometry']['Lrt']

In [34]:
from joblib import Parallel, delayed

In [35]:
#single parameter function for parallelization
def getPert(l):
    opt = SixMirror(thetOpening = 20, Rfast = 50e-3, Rplane = 1e5, Rslow = -250e-3, diam=6e-3, dy=8e-3, dx=l)
    system = geometry(**opt)
    sp, s = calcPertFast(system, chi)
    fsr = 3.0e8/system['geometry']['Lrt']
    return sp, s, fsr

In [43]:
result = Parallel(n_jobs=8)(delayed(getPert)(l) for l in Las)

specPert, spec, fsrs = zip(*result)
specPert = np.stack(specPert, axis=0)
spec = np.stack(spec, axis=0)
fsrs = np.stack(fsrs, axis=0)

In [44]:
smhz = specPert*fsrs[:,None]/(2*np.pi)
fsrc = fsrs[fsrs.shape[0]//2]
fplt = np.mod(3*smhz+fsrc/2, fsrs[:,None])

f, ax = plt.subplots(ncols=2, figsize=(9,4))
ax[0].plot(Las, fplt, '.', ms=2)
#plt.plot(Las[:], np.mod(3* smhz[:,::3]+fsr/2, fsrs[:,None]), '.', ms=2)
#plt.plot(Las, specPert, '.', ms=2)
std =  np.std(fplt[:,2:-2],axis=1)
ax[1].plot(Las, std)
minimum = np.argmin(std)
ax[0].axvline(Las[minimum], color='r')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
%timeit calcPert(system, chi)

100 loops, best of 5: 14.4 ms per loop


In [40]:
%lprun -f calcPert calcPert(system, chi)

Timer unit: 1e-07 s

Total time: 0.019161 s
File: <ipython-input-31-a3315af079d0>
Function: calcPert at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calcPert(system, chi):
     2         1         70.0     70.0      0.0      a, b, aD, bD = chi
     3         1       3996.0   3996.0      2.1      MuAndChi = getEvN(system['M'])
     4         1         42.0     42.0      0.0      xsi1, xsi2 = MuAndChi['chi']
     5                                               
     6         1          6.0      6.0      0.0      mu = MuAndChi['mu']
     7         1        509.0    509.0      0.3      InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
     8         1      33914.0  33914.0     17.7      Upar = expm(1j*(xsi1*aD@a + xsi2*bD@b))
     9         1       8318.0   8318.0      4.3      XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, InvEigs.T, chi)
    10                                           

In [57]:
%timeit calcPertFast(system, chi)

100 loops, best of 5: 5.94 ms per loop


In [195]:
%lprun -f calcPertFast calcPertFast(system, chi)

Timer unit: 1e-07 s

Total time: 152.908 s
File: <ipython-input-181-ee0916e98325>
Function: calcPertFast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calcPertFast(system, chi):
     2         1       3414.0   3414.0      0.0      a, b, aD, bD = chi
     3         1     343523.0 343523.0      0.0      MuAndChi = getEvN(system['M'])
     4         1         71.0     71.0      0.0      xsi1, xsi2 = MuAndChi['chi']
     5                                               
     6         1          6.0      6.0      0.0      mu = MuAndChi['mu']
     7         1       1861.0   1861.0      0.0      InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
     8         1   16689628.0 16689628.0      1.1      Argpar = (xsi1*aD@a + xsi2*bD@b) # argument of Upar = expm(1j*Argpar), is diagonal anyways!
     9         1   48248965.0 48248965.0      3.2      XPrefMat = np.sqrt(2/k)*np.einsum("ji,jk,kxy->ixy", G, I

In [209]:
%lprun -f calcPertSparse calcPertSparse(system, chi_sp)

Timer unit: 1e-07 s

Total time: 47.5199 s
File: <ipython-input-208-ecdbe7c624b5>
Function: calcPertSparse at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calcPertSparse(system, chi):
     2         1      18027.0  18027.0      0.0      a, b, aD, bD = chi
     3         1       3476.0   3476.0      0.0      MuAndChi = getEvN(system['M'])
     4         1         45.0     45.0      0.0      xsi1, xsi2 = MuAndChi['chi']
     5                                               
     6         1          7.0      7.0      0.0      mu = MuAndChi['mu']
     7         1        453.0    453.0      0.0      InvEigs = np.linalg.inv(np.concatenate([mu, mu.conj()], axis=1))
     8         1        975.0    975.0      0.0      G_sp = sparse.COO.from_numpy(G)
     9         1        836.0    836.0      0.0      InvEigs_sp = sparse.COO.from_numpy(InvEigs.T)
    10                                               
    11         1